# Análise de Sentimentos - IMDB

# Dataset Analysis

In [1]:
def pretty_print_review_and_label(i):
    print(labels[i] + "\t:\t" + reviews[i][:80] + "...")

g = open('dataset/reviews.txt','r')
reviews = list(map(lambda x:x[:-1], g.readlines()))
g.close()

g = open('dataset/labels.txt','r')
labels = list(map(lambda x:x[:-1].upper(), g.readlines()))
g.close()

**Nota:** Os dados em `reviews.txt` que estamos utilizando já passaram por algumas etapas de pré-processamento e contêm apenas caracteres em letras minúsculas. Isso deve ser feito para podermos lidar com palavras iguais escritas de formas diferentes, como `The`, `the` e `THE`, que são convertidas para `the`.

In [2]:
len(reviews)

25000

In [3]:
reviews[0]

'bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   '

In [4]:
labels[0]

'POSITIVE'

# 1. Processamento do Texto

In [5]:
import pandas as pd

In [6]:
df = pd.DataFrame({'REVIEW': reviews, "LABEL": labels})

In [7]:
df.head()

,REVIEW,LABEL
0,bromwell high is a cartoon comedy . it ran at ...,POSITIVE
1,story of a man who has unnatural feelings for ...,NEGATIVE
2,homelessness or houselessness as george carli...,POSITIVE
3,airport starts as a brand new luxury pla...,NEGATIVE
4,brilliant over acting by lesley ann warren . ...,POSITIVE


## 1.1. Pontuação, Stop Words, Lemmatization

In [8]:
import re
import unicodedata
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import RSLPStemmer
from sklearn.model_selection import train_test_split
import torch
from torchtext import data
import torch.nn as nn

In [11]:
wnl = WordNetLemmatizer()
stemmer = RSLPStemmer()
BAD_SYMBOLS_RE = re.compile(r'[^0-9a-z]')
STOPWORDS = set(stopwords.words('english'))

In [12]:
print(f"lemmatize: {wnl.lemmatize('fetched')}")
print(f"stemmer: {stemmer.stem('fetched')}")

lemmatize: fetched
stemmer: fetched


In [13]:
def strip_accents(text):
    try:
        text = unicode(text, 'utf-8')
    except (TypeError, NameError):  # unicode is a default on python 3
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")

    return str(text)

In [14]:
def preprocess_text_stop(text):
    text = text.lower()
    bad_symbols_re = re.compile(r'[^0-9a-z]')
    text = strip_accents(text)
    text = bad_symbols_re.sub(' ', text)
    text = ' '.join(word for word in text.split() if word not in set(stopwords.words('english')))

    return text

In [15]:
def preprocess_lemm_text(text):
    text = text.lower()
    
    text = strip_accents(text)
    text = BAD_SYMBOLS_RE.sub(' ', text)
    text = ' '.join(wnl.lemmatize(word)
                    for word in text.split() if word not in STOPWORDS)

    return text

In [16]:
df['REVIEW_P'] = df['REVIEW'].apply(preprocess_lemm_text)

## 1.2. Visualizando resultado

In [17]:
df.iloc[0][2]

'bromwell high cartoon comedy ran time program school life teacher year teaching profession lead believe bromwell high satire much closer reality teacher scramble survive financially insightful student see right pathetic teacher pomp pettiness whole situation remind school knew student saw episode student repeatedly tried burn school immediately recalled high classic line inspector sack one teacher student welcome bromwell high expect many adult age think bromwell high far fetched pity'

In [18]:
df.iloc[0][0]

'bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   '

In [19]:
df.head()

,REVIEW,LABEL,REVIEW_P
0,bromwell high is a cartoon comedy . it ran at ...,POSITIVE,bromwell high cartoon comedy ran time program ...
1,story of a man who has unnatural feelings for ...,NEGATIVE,story man unnatural feeling pig start opening ...
2,homelessness or houselessness as george carli...,POSITIVE,homelessness houselessness george carlin state...
3,airport starts as a brand new luxury pla...,NEGATIVE,airport start brand new luxury plane loaded va...
4,brilliant over acting by lesley ann warren . ...,POSITIVE,brilliant acting lesley ann warren best dramat...


## 1.3. Removendo coluna Review

In [20]:
df.drop(columns=['REVIEW'], inplace=True)

## 1.4. Renomeando as colunas

In [22]:
df.rename(columns={"LABEL": "target", "REVIEW_P": "text"}, inplace=True)

## 1.5. Separando os dados em Treino, Teste e Validação

In [23]:
train_df, test_df = train_test_split(df, test_size = 0.2)


In [24]:
train_df.shape

(20000, 2)

In [25]:
test_df.shape

(5000, 2)

In [26]:
test_df, valid_df = train_test_split(test_df, test_size = 0.5)

In [27]:
test_df.shape

(2500, 2)

In [28]:
valid_df.shape

(2500, 2)

In [29]:
train_df.head()

,target,text
16429,NEGATIVE,indeed quite strange movie first ex u gymnast ...
18450,POSITIVE,mistake war inc sharply chiseled satire brainy...
20,POSITIVE,first read armistead maupins story taken human...
8884,POSITIVE,good picture worth word film poetic scene ever...
20624,POSITIVE,one military drama like lot tom berenger playi...


In [30]:
df.isna().sum()

target    0
text      0
dtype: int64

## 1.6. Métodos para criar o Dataset a partir do Dataframe com 2 colunas

In [32]:
class DataFrameDataset(data.Dataset):

    def __init__(self, df, fields, is_test=False, **kwargs):
        examples = []
        for i, row in df.iterrows():
            label = row.target if not is_test else None
            text = row.text
            examples.append(data.Example.fromlist([text, label], fields))

        super().__init__(examples, fields, **kwargs)

    @staticmethod
    def sort_key(ex):
        return len(ex.text)

    @classmethod
    def splits(cls, fields, train_df, val_df=None, test_df=None, **kwargs):
        train_data, val_data, test_data = (None, None, None)
        data_field = fields

        if train_df is not None:
            train_data = cls(train_df.copy(), data_field, **kwargs)
        if val_df is not None:
            val_data = cls(val_df.copy(), data_field, **kwargs)
        if test_df is not None:
            test_data = cls(test_df.copy(), data_field, **kwargs)

        return tuple(d for d in (train_data, val_data, test_data) if d is not None)

# 2. BERT

Vamos utilizar o modelo de BERT, que é um modelo transformador. Os modelos de transformadores são muito maiores do que modelos simples de Deep Learning para NLP (LSTM, CNN, etc). Utilizaremos a biblioteca ```transformers``` para obter o modelo BERT pré-treinado. Vamos utilizar a técnica de transfer learning e congelaremos o trasformador (maior parte do modelo), vamos treinar "apenas" uma GRU bidirecional e uma camada Densa de saída.

Para instalar a biblioteca:
```bash
! pip install transformers
```

In [45]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

## 2.1. Tokenizador

In [46]:
len(tokenizer.vocab)

30522

O transformador possui tokens especiais para marcar o início e o final da frase. Bem como um preenchimento padrão e um token desconhecido.

In [47]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


Podemos obter os índices dos tokens especiais utilizando o método ```convert_tokens_to_ids```

In [48]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


Outra forma de obte-los é diretamente do tokenizer

In [49]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


O tokenizer do BERT foi treinado com sequências de tamanho fixo (512 tokens). Podemos obter o tamanho máximo utilizando o método ```ax_model_input_sizes```

In [50]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length)



512


Função para converter a o nosso campo TEXT para o padrão aceito pelo BERT. Observe que nosso comprimento máximo é 2 itens menores que o comprimento de 512 tokens. Isso tem que ser feito porque o processo de tokenizar adiciona dois tokens a cada sequência (um no início e outro do final)

In [51]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

Definimos o campo do rótulo como antes.

Agora definimos nossos campos de TEXT e LABEL: 
- O BERT espera que a dimensão do lote seja a primeira (```batch_first = True```);
- Vamos utilizar o vocabulário do BERT (```use_vocab = False```);
- Passamos a nossa função (```tokenize = tokenize_and_cut```);
- O argumento de preprocessing é uma função que converte os tokens em índices (```preprocessing = tokenizer.convert_tokens_to_ids```);
- Definimos os tokens especiais

In [52]:
from torchtext import data

TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

In [53]:
fields = [('text',TEXT), ('label',LABEL)]

train_ds, val_ds, test_ds = DataFrameDataset.splits(fields, train_df=train_df, val_df=valid_df, test_df=test_df)

In [54]:
train_data = train_ds
test_data = test_ds 
valid_data = val_ds

In [56]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 20000
Number of validation examples: 2500
Number of testing examples: 2500


Podemos verificar um exemplo e garantir que o texto já tenha sido numerado.

In [57]:
print(vars(train_data.examples[6]))

{'text': [3185, 2919, 4133, 2302, 2242, 2842, 5436, 2391, 12246, 11471, 2143, 3233, 9971, 9131, 2028, 8257, 6057, 2240, 11007, 6412, 3761, 25869, 2964, 2050, 4092, 4087, 15723, 7446, 2071, 3393, 14036, 7987, 7987, 2081, 3737, 2491, 2143, 3666, 2611, 20345, 14836, 9202, 3325, 7987, 7987, 2524, 5674, 4205, 5472, 3917, 2468, 2759, 2596, 2986, 4038, 2583, 5788, 2785, 7524, 6057, 2560, 3185, 11268, 3015, 8995, 4621, 4038], 'label': 'NEGATIVE'}


Podemos usar o convert_ids_to_tokens para transformar esses índices novamente em tokens legíveis.

In [58]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['text'])

print(tokens)

['movie', 'bad', 'sit', 'without', 'something', 'else', 'plot', 'point', 'thoroughly', 'bored', 'film', 'stand', 'comedian', 'recall', 'one', 'joke', 'funny', 'line', 'worthy', 'description', 'politician', 'char', '##ism', '##a', 'speaking', 'technical', 'jar', '##gon', 'could', 'le', 'entertaining', 'br', 'br', 'made', 'quality', 'control', 'film', 'watching', 'girl', 'bikini', 'distraction', 'horrible', 'experience', 'br', 'br', 'hard', 'imagine', 'adam', 'sand', '##ler', 'become', 'popular', 'appeared', 'fine', 'comedy', 'able', 'survive', 'kind', 'exposure', 'funny', 'least', 'movie', 'prof', 'writing', 'vital', 'effective', 'comedy']


Criando o vocabulário para os rótulos.

In [59]:
LABEL.build_vocab(train_data)

In [60]:
print(LABEL.vocab.stoi)

defaultdict(None, {'POSITIVE': 0, 'NEGATIVE': 1})


Vamos criar os iteradores. Para os modelos transformadores, quanto maior o BATCH_SIZE, melhor o resultado. 

In [61]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

## 3. Build model

In [62]:
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

A seguir, definiremos nosso modelo atual.

Em vez de usar uma camada de embedding para obter os embeddings para o nosso texto, usaremos o modelo BERT pré-treinado. Esses embeddings serão então alimentados em uma GRU para produzir uma previsão do sentimento da sentença de entrada. Obtemos o tamanho da dimensão dos embedding (chamado hidden_size) do BERT por meio de seu atributo de configuração. O restante da inicialização é padrão.

Na forward, envolvemos o transformador em um no_grad para garantir que nenhum gradiente seja calculado nessa parte do modelo. O transformador realmente retorna os embeddings para toda a sequência, bem como uma saída em pool. O restante do forward é a implementação padrão de um modelo recorrente, onde pegamos o estado oculto durante o passo final do tempo e passamos por uma camada linear para obter nossas previsões.

In [63]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output



Em seguida, criamos uma instância do nosso modelo.

In [64]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

Podemos verificar quantos parâmetros o modelo possui (112 milhões!). Felizmente, 110M desses parâmetros são do transformador e não iremos treiná-los.

In [65]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 112,241,409 trainable parameters


Para congelar os parâmetros (não treiná-los), precisamos definir o atributo require_grad como False. Para fazer isso, simplesmente percorremos todos os parâmetros named_ em nosso modelo e, se eles fazem parte do modelo do transformador bert, configuramos o require_grad = False.

In [66]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False



Agora podemos ver que nosso modelo possui parâmetros treináveis da 3M.

In [67]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')



The model has 2,759,169 trainable parameters


Podemos verificar os nomes dos parâmetros treináveis, garantindo que eles façam sentido. Como podemos ver, eles são todos os parâmetros da GRU (rnn) e da camada linear (out).

In [68]:
for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l0_reverse
rnn.weight_hh_l0_reverse
rnn.bias_ih_l0_reverse
rnn.bias_hh_l0_reverse
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
rnn.weight_ih_l1_reverse
rnn.weight_hh_l1_reverse
rnn.bias_ih_l1_reverse
rnn.bias_hh_l1_reverse
out.weight
out.bias


## 4.Train the model

Definimos nosso otimizador e função de perda.

In [69]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

Verificando CUDA

In [70]:
model = model.to(device)
criterion = criterion.to(device)

A seguir, definiremos funções para: calcular acurácia, executar uma época de treinamento, executar uma época de avaliação e calcular quanto tempo leva uma época de treinamento / avaliação.

In [71]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [72]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [73]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [74]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Finalmente, treinaremos nosso modelo. Mesmo não treinando nenhum dos parâmetros do transformador, ainda precisamos passar os dados pelo modelo, o que leva uma quantidade considerável de tempo.

In [75]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '/model/tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 342m 39s
	Train Loss: 0.606 | Train Acc: 66.15%
	 Val. Loss: 0.428 |  Val. Acc: 80.34%
Epoch: 02 | Epoch Time: 277m 57s
	Train Loss: 0.461 | Train Acc: 78.16%
	 Val. Loss: 0.486 |  Val. Acc: 78.40%
Epoch: 03 | Epoch Time: 277m 27s
	Train Loss: 0.391 | Train Acc: 82.58%
	 Val. Loss: 0.343 |  Val. Acc: 84.81%
Epoch: 04 | Epoch Time: 277m 14s
	Train Loss: 0.366 | Train Acc: 83.78%
	 Val. Loss: 0.392 |  Val. Acc: 84.10%
Epoch: 05 | Epoch Time: 301m 30s
	Train Loss: 0.339 | Train Acc: 85.44%
	 Val. Loss: 0.338 |  Val. Acc: 85.81%


Carregaremos os parâmetros que obtemos o menor valor de loss em validação e utilizamos o modelo no conjunto de testes.

In [76]:
model.load_state_dict(torch.load('tut6-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.309 | Test Acc: 87.64%


## Inferência

Em seguida, usaremos o modelo para testar o sentimento de algumas seqüências. Nós tokenizamos a sequência de entrada, a reduzimos para o comprimento máximo, adicionamos os tokens especiais a ambos os lados, convertemos em um tensor, adicionamos uma dimensão de lote falsa e passamos pelo modelo.

In [77]:
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [80]:
predict_sentiment(model, tokenizer, "This film is terrible and bad")

0.9718824625015259

In [81]:
predict_sentiment(model, tokenizer, "This film is great")

0.01686229556798935

### FIM